In [12]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [13]:
#FUNCIÓN PARA TRADUCIR

def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated

#FUNCIÓN PARA LIMPIAR

def cleaning(text):
    text = text.lower()
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    return doc

In [14]:
#BUSCAR EN EDUCACIÓN
df = pd.read_excel("3_final_db.xlsx")
# dfe = df[(df["search"]=="educ_regex") | (df["search"]=="educ_fp") | (df["search"]=="educ_fs")]

dfe = df[df["education"].notna()]
dfe["education"] =  dfe["education"].str.lower()

<ipython-input-14-2b2aacda90a5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["education"] =  dfe["education"].str.lower()


In [15]:
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]


K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = int(c["start"])

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = int(c["end"])

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

dfe["educacion"] = K

<ipython-input-15-f109f040ecc8>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["educacion"] = K


In [16]:
dfe.reset_index(drop = True, inplace = True)

In [17]:
#SELECCIONAR A LOS UNIVERSITARIOS SOLO SI ESTUDIARON MÁS DE 2 AÑOS
H = []

for i in range(len(dfe)):
    educacion_ind = [] 
    for e in range(len(dfe["educacion"][i])):
        # k = []
        if "start" in dfe["educacion"][i][e].keys() and "end" in dfe["educacion"][i][e].keys():
            if type(dfe["educacion"][i][e]["start"]) == int and type(dfe["educacion"][i][e]["end"]) == int:
                resta = dfe["educacion"][i][e]["end"] - dfe["educacion"][i][e]["start"]
                if resta >=3:
                    educacion_ind.append(dfe["educacion"][i][e])
        # educacion_ind.append(k)
    H.append(educacion_ind)


In [18]:
#TRADUCIR

for i in tqdm(H):
    for e in tqdm(i):
        if "degrees" in e and len(e["degrees"])>0:
           try:
               e["degrees"] = translateString(str(e["degrees"]), "en")
               e["degrees"] = cleaning(e["degrees"])
           except:
              None

        if "education" in tqdm(e) and len(e["education"])>0:
           try:
            e["education"] = translateString(str(e["education"]), "en")
            e["education"] = cleaning(e["education"])
           except:
              None

100%|██████████| 1/1 [00:00<00:00,  1.03it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.06it/s]
0it [00:00, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.07it/s]
0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.53s/it]
0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.54s/it]
0it [00:00, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.43s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.00s/it]
0it [00:00, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.13s/it]
0it [00:00, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]
0it [00:00, ?it/s]

  0%|          | 

In [29]:
for i in H:
    for e in i:
        if "degrees" in e and len(e["degrees"])>0:
            e["degrees"] = str(e["degrees"])

        if "education" in tqdm(e) and len(e["education"])>0:
            e["education"] = str(e["education"])


  0%|          | 0/5 [00:00<?, ?it/s]


In [30]:
#loop para los grados

#GRADOS
sp = ["specialisation", "specialization"]
ms = ["master", "msc ", "m.a. ", "m.s. ", "mba ", "mphill ", "mres ", "mag ", " msc", " m.a.", " m.s.", " mba", " mphill", " mres", " mag"]
ph = ["phd", "dr ", "dr phil"]


#FUNCION PARA ENCONTRAR GRADO
def degree(palabra):
    u = "bs"
    for e in ph:
        if e in palabra:
            u =  "phd"

    for k in ms:
        if k in palabra:
            u = "ms"

    for t in sp:
        if t in palabra:
            u = "sp"
    return u


In [31]:
#BACHILLER Y NO U 

B = H.copy()
for i in range(len(H)):
    # B[i] = H[i]
    for e in range(len(H[i])):
        if "degrees" in H[i][e] and len(H[i][e]["degrees"])>0:
            B[i]= str(degree(H[i][e]["degrees"]))
        else:
            B[i]= "bs"

for i in range(len(B)):
    if type(B[i]) == list:
        B[i] = "NO_U"
    

In [32]:
#BASE FINAL
dfe["degree"] = B
dfe = dfe[["id","degree"]]

<ipython-input-32-3d58a4acb857>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["degree"] = B


In [ ]:
final_db = pd.merge(df, dfe, "left", "id")

In [ ]:
final_db.to_excel("4_final_db.xlsx", index = False)